In [1]:
## Biermann Deck Maker
## (c) F. S. Tsung 2019

##
## library and constants are here
## What I do, and what I recommend that you do as well is to use Github to load my library, and include 
## it in the path as shown below.  
##
## the constants are in CGS units, I suggest that you do the same.  The notebook assumes CGS units.
##

import sys
# on my mac pro
sys.path.append('/Users/uclapic/Documents/code-development/python-tsung') # go to parent dir
# on my laptop
# sys.path.append('c:/Users/Frank Tsung/Documents/GitHub/python-tsung')


# import hfhi
import numpy as np
import hfhi

me = 9.1095e-28
qe = 4.8032e-10
c_cgs = 2.998e10

mypi = 3.1415926

#### 2D Biermann Deck Maker



In [2]:
## first some common constants 




In [3]:
## enter the basic
##



## in the simulations, we adopt the normalization that the density is normalized to n0, speed is normalized to c, and q/m for 
## electrons is -1.  Therefore, under these conditions, 
## nppc -> number of particles per cell

## In our normalizations, q/m for electrons is -1
## n0 is 1
## so electron plasma frequency is 1 by definition
## and speed of light is 1
## so length 

nppc = 64

n0 = 1.0

nb = 0.1

vth0 = 0.2

vb = 0.01

Ln = 20

Lt = 40


length = 15/8 * Lt * 2

print('System Length =',repr(length))

duration = 1300

print('Total Duration = ', repr(duration),'\omega_{pe}')


('System Length =', '80')
('Total Duration = ', '1300', '\\omega_{pe}')


In [4]:

# let n0 = 1, so omega_pe = 1.

omega_pe = np.sqrt(n0)
debye_length= vth0 / omega_pe 

# print(debye_length)


resolution = 0.9 * debye_length

grid_number = length/resolution

# for 2D it's sqrt(1/2), for 3D the stepsize is sqrt(1/3)
# due to the CFL condition of the FDTD scheme.
#
#

CFL = 0.98

timestep = CFL * np.sqrt(0.5) * resolution

total_cells = int(length/resolution*length/resolution)

total_steps = duration / timestep

num_par = total_cells * nppc

total_cpus = 4

# ------------------------------------------------------------------
# --  do not touch the lines below, the magic is happening below ---
# ------------------------------------------------------------------



num_par_max = 1.25 * num_par/total_cpus


print('Length in normalized units is : ' +repr(length))
print('Grid size in normalized unit is : '+repr( resolution ))
print('total number of cells = '+repr(total_cells))
print('total number of particles = '+repr(total_cells*nppc/1e6)+' millions')
print('total number of timesteps = '+repr(total_steps))
print('CPU estimate = '+repr(total_steps*total_cells*nppc*2e-7/3600)+' hours')
print('num_par_max estimate = '+repr(num_par_max))

Length in normalized units is : 80
Grid size in normalized unit is : 0.18000000000000002
total number of cells = 197530
total number of particles = 12.64192 millions
total number of timesteps = 10422.208792998999
CPU estimate = 7.3198183213549948 hours
num_par_max estimate = 3950600.0


In [5]:
## Here we work out the simulation box size and so on.
##
##
import math
##


##
##
## here we describe the node layout
##
##
nprocx = 4
nprocy = 4


## here we describe the time between diagnostics
dump_time = 100.0 
ndump = math.ceil(dump_time/timestep)
##
##
print("node_conf")
print("{")
print("  nx_p(1:2) = ", repr(nprocx),',', repr(nprocy),',' )
print(" coordinates = \"cartesian \", ")
print(" if_periodic(1:2) = .true., .true.,")
print("}")

print("grid")
print("{")
print("  nx_p(1:2) = ", repr(math.ceil(grid_number)),',', repr(math.ceil(grid_number)),',' )
print("}")


print("time_step")
print("{")
print("  dt = ", repr(timestep),',')
print("  ndump = ", repr(ndump),',')
print("}")

print("restart")
print("{")
print("if_restart = .false.,")
print("ndump_fac = 0 ,")
print("}")





print("space")
print("{")
print("  xmin(1:2) = ", repr(-length/2.0),',', repr(-length/2.0),',' )
print("  xmax(1:2) = ", repr(length/2.0),',', repr(length/2.0),',' )
print("}")

print("time")
print("{")
print("  tmin = 0 ," )
print("  tmax = ", repr(duration),',' )
print("}")

node_conf
{
('  nx_p(1:2) = ', '4', ',', '4', ',')
 coordinates = "cartesian ", 
 if_periodic(1:2) = .true., .true.,
}
grid
{
('  nx_p(1:2) = ', '445.0', ',', '445.0', ',')
}
time_step
{
('  dt = ', '0.12473363620130699', ',')
('  ndump = ', '802.0', ',')
}
restart
{
if_restart = .false.,
ndump_fac = 0 ,
}
space
{
('  xmin(1:2) = ', '-40.0', ',', '-40.0', ',')
('  xmax(1:2) = ', '40.0', ',', '40.0', ',')
}
time
{
  tmin = 0 ,
('  tmax = ', '1300', ',')
}


In [10]:
## Here we make the namelist for electrons

## electron namelist


print('species ')
print(' {')
# print(' num_par_max = '+repr(int(num_par_max))+' ,')
print(' rqm = -1.0 , ')
print(' num_par_x(1,2) = 8,8, ')
print(' n_sort = 25, ')
print('}')

print('udist')
print('{')

print('  use_spatial_uth=.true.,')


# print('spatial_uth(1)=')
# print(' spatial_ufl(1) = (if (x1*x1+x2*x2 <',repr(Lt*Lt),') ,',repr(vth0+vb),', ' )
print('  spatial_ufl(1) = (if (x1*x1  <',repr(Lt*Lt),') ,',repr(vth0-vb),
      ' * cos(3.1415926*x1/(2.0 *',repr(Lt),') ) + '
      ,repr(vb),', ',repr(vb),'),')
print('  spatial_ufl(2) = (if (x1*x1  <',repr(Lt*Lt),') ,',repr(vth0-vb),
      ' * cos(3.1415926*x1/(2.0 *',repr(Lt),') ) + '
      ,repr(vb),', ',repr(vb),'),')
print('  spatial_ufl(3) = (if (x1*x1  <',repr(Lt*Lt),') ,',repr(vth0-vb),
      ' * cos(3.1415926*x1/(2.0 *',repr(Lt),') ) + '
      ,repr(vb),', ',repr(vb),'),')



print('profile')
print('{')
print('  profile_type(1,2)= \"math func\", \"math func\"' )
print('  math_func_expr = (if (sqrt(x1*x1 + (x2*x2*',repr(Lt/Ln*Lt/Ln),' )) < ',repr(Lt) , 
      '),  (',
      repr(n0),'-',repr(nb),
      ') * cos( 3.1415926 * sqrt(x1*x1+(x2*x2)*',repr(Lt/Ln*Lt/Ln),')/ ( 2*',repr(Lt),' ) ), ' 
      ,repr(nb))
# print(' density = '+ repr(density)+' ,')
print('}')

print('spe_bound')
print('{')

print('}')


species 
 {
 rqm = -1.0 , 
 num_par_x(1,2) = 8,8, 
 n_sort = 25, 
}
udist
{
  use_spatial_uth=.true.,
('  spatial_ufl(1) = (if (x1*x1  <', '1600', ') ,', '0.19', ' * cos(3.1415926*x1/(2.0 *', '40', ') ) + ', '0.01', ', ', '0.01', '),')
('  spatial_ufl(2) = (if (x1*x1  <', '1600', ') ,', '0.19', ' * cos(3.1415926*x1/(2.0 *', '40', ') ) + ', '0.01', ', ', '0.01', '),')
('  spatial_ufl(3) = (if (x1*x1  <', '1600', ') ,', '0.19', ' * cos(3.1415926*x1/(2.0 *', '40', ') ) + ', '0.01', ', ', '0.01', '),')
profile
{
  profile_type(1,2)= "math func", "math func"
('  math_func_expr = (if (sqrt(x1*x1 + (x2*x2*', '4', ' )) < ', '40', '),  (', '1.0', '-', '0.1', ') * cos( 3.1415926 * sqrt(x1*x1+(x2*x2)*', '4', ')/ ( 2*', '40', ' ) ), ', '0.1')
}
spe_bound
{
}


In [6]:
## ion namelist:

## Here we make the namelist for electrons

## electron namelist
rmass = 400

vth_i = 0.01

print('species ')
print(' {')
# print(' num_par_max = '+repr(int(num_par_max))+' ,')
print(' rqm = ',repr(rmass),',')
print(' num_par_x(1,2) = 8,8, ')
print(' n_sort = 25, ')
print('}')

print('udist')
print('{')

print('  ')


# print('spatial_uth(1)=')
# print(' spatial_ufl(1) = (if (x1*x1+x2*x2 <',repr(Lt*Lt),') ,',repr(vth0+vb),', ' )
print('  uth(1) = ' ,repr(vth_i), '),')
print('  uth(2) = ' ,repr(vth_i), '),')
print('  uth(3) = ' ,repr(vth_i), '),')


print('profile')
print('{')
print('  profile_type(1,2)= \"math func\", \"math func\"' )
print('  math_func_expr = (if (sqrt(x1*x1 + (x2*x2*',repr(Lt/Ln*Lt/Ln),' )) < ',repr(Lt) , 
      '),  (',
      repr(n0),'-',repr(nb),
      ') * cos( 3.1415926 * sqrt(x1*x1+(x2*x2)*',repr(Lt/Ln*Lt/Ln),')/ ( 2*',repr(Lt),' ) ), ' 
      ,repr(nb))
# print(' density = '+ repr(density)+' ,')
print('}')

print('spe_bound')
print('{')

print('}')


species 
 {
(' rqm = ', '400', ',')
 num_par_x(1,2) = 8,8, 
 n_sort = 25, 
}
udist
{
  
('  uth(1) = ', '0.01', '),')
('  uth(2) = ', '0.01', '),')
('  uth(3) = ', '0.01', '),')
profile
{
  profile_type(1,2)= "math func", "math func"
('  math_func_expr = (if (sqrt(x1*x1 + (x2*x2*', '4', ' )) < ', '40', '),  (', '1.0', '-', '0.1', ') * cos( 3.1415926 * sqrt(x1*x1+(x2*x2)*', '4', ')/ ( 2*', '40', ' ) ), ', '0.1')
}
spe_bound
{
}
